<a href="https://colab.research.google.com/github/mkschulz9/fine-tuning-stable-diffusion-3/blob/main/oscar_lora_finetuning_webshard_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata

repo_name = "fine-tuning-stable-diffusion-3"
username = "mkschulz9"
repo_token = userdata.get("REPO_TOKEN")

# Clone private repository
if not os.path.exists(f'/content/{repo_name}'):
    !git clone https://{repo_token}@github.com/{username}/{repo_name}.git
    %cd {repo_name}
else:
    %cd {repo_name}
    !git pull

Cloning into 'fine-tuning-stable-diffusion-3'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 244 (delta 9), reused 16 (delta 6), pack-reused 220 (from 1)
Receiving objects: 100% (244/244), 2.72 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (115/115), done.
/content/fine-tuning-stable-diffusion-3


**Folders:**

*   webdataset_shards_train: webdataset folder containing train data as webdataset
*   List item



In [ ]:
!git checkout -b lora-oscar

Switched to a new branch 'lora-oscar'


In [ ]:
!git clone https://github.com/gojasper/flash-diffusion.git

Cloning into 'flash-diffusion'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 188 (delta 27), reused 58 (delta 22), pack-reused 123 (from 1)
Receiving objects: 100% (188/188), 51.60 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
#Check Python version is 3.10
!python --version

Python 3.10.12


In [ ]:
!pip install --upgrade pip
!pip install -r /content/fine-tuning-stable-diffusion-3/flash-diffusion/requirements.txt
!pip install -e /content/fine-tuning-stable-diffusion-3/flash-diffusion/.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Cloning https://github.com/initml/diffusers.git (to revision clement/feature/flash) to /tmp/pip-install-ya67bi2n/diffusers_4ac854bc3fa44bd7ac2939f9c232c723
  Running command git clone --filter=blob:none --quiet https://github.com/initml/diffusers.git /tmp/pip-install-ya67bi2n/diffusers_4ac854bc3fa44bd7ac2939f9c232c723
  Running command git checkout -b clement/feature/flash --track origin/clement/feature/flash
  Switched to a new branch 'clement/feature/flash'
  Branch 'clement/feature/flash' set up to track remote branch 'clement/feature/flash' from 'origin'.
  Resolved https://github.com/initml/diffusers.git to commit 04bb8ffc1df6dd85427e2508fd8582a57e984787
  Installing build depende

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/')

In [ ]:
import datasets
dataset = datasets.load_from_disk('pokemon_cards_train_test_split')

In [ ]:
val_test_datasplit = dataset['test'].shuffle().train_test_split(test_size=0.5)

In [ ]:
from datasets import DatasetDict

val_test_datasplit = DatasetDict(
    {
        "val": val_test_datasplit["train"],
        "test": val_test_datasplit["test"],
    }
)

In [ ]:
val_test_datasplit.save_to_disk('/content/pokemon_cards_validation_test_split')

Saving the dataset (0/1 shards):   0%|          | 0/1314 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1314 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
import sys
sys.path.append('/content/fine-tuning-stable-diffusion-3/data_preprocessing')
import data_preprocessor

# Load and preprocess HuggingFace dataset
dataset = data_preprocessor.DataPreprocessor("TheFusion21/PokemonCards")
dataset.reduce_captions()
dataset_split = dataset.remove_columns_split_dataset(column_names=["name", "hp", "set_name"])


Dataset loaded: TheFusion21/PokemonCards

***REDUCING CAPTIONS TO FIRST SENTENCE***

***REMOVING COLUMNS & SPLITTING DATASET***
Columns removed and dataset successfully split: DatasetDict({
    train: Dataset({
        features: ['id', 'image_url', 'caption'],
        num_rows: 10511
    })
    test: Dataset({
        features: ['id', 'image_url', 'caption'],
        num_rows: 2628
    })
})


In [ ]:
val_test_datasplit.save_to_disk('/content/pokemon_cards_val_test_split')

Saving the dataset (0/1 shards):   0%|          | 0/1314 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1314 [00:00<?, ? examples/s]

In [ ]:
import json, os, webdataset
from PIL import Image
from io import BytesIO
import requests

#Directory for WebDataset shards format
os.makedirs('/content/webdataset_shards_train', exist_ok=True)

#Convert image to bytes
def image_to_bytes(image):
  image_buffer = BytesIO()
  image = image.convert('RGB')
  image.save(image_buffer, format='JPEG')
  return image_buffer.getvalue()

In [ ]:
# Write samples to WebDataset shards
shard_size = 1000
shard_id = 0
samples_in_shard = []

for idx, sample in enumerate(dataset["train"]):
    #Retrieve image from image_url
    image_url = sample['image_url']
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))

    #Retrieve image caption
    caption = sample['caption']

    # Convert the image to JPEG bytes
    image_bytes = image_to_bytes(image)

    # Create a JSON metadata file
    metadata = {
        "caption": caption,
        "aesthetic_score": 6.0,
    }
    json_bytes = json.dumps(metadata).encode("utf-8")

    # Save the sample
    samples_in_shard.append({
        "__key__": f"{idx:06d}",
        "jpg": image_bytes,
        "json": json_bytes,
    })

    # Write to shard when full
    if len(samples_in_shard) >= shard_size:
        shard_path = os.path.join('/content/webdataset_shards_train', f"shard_{shard_id:05d}.tar")
        with webdataset.TarWriter(shard_path) as writer:
            for sample in samples_in_shard:
                writer.write(sample)
        shard_id += 1
        samples_in_shard = []

# Write remaining samples
if samples_in_shard:
    shard_path = os.path.join('/content/webdataset_shards_train', f"shard_{shard_id:05d}.tar")
    with webdataset.TarWriter(shard_path) as writer:
        for sample in samples_in_shard:
            writer.write(sample)

In [ ]:
#Move files from folder under /content to google drive
import shutil

source_folder = '/content/webdataset_shards_train'
destination_folder = '/content/drive/MyDrive/webdataset_shards_train'  # Change 'MyDrive/webdataset_shards' to your desired path in Google Drive

shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)

'/content/drive/MyDrive/webdataset_shards_test'

In [ ]:
!python3.10 /content/flash-diffusion/examples/train_flash_sd3.py